In [1]:
import sagemaker as sage
import pandas as pd
from time import gmtime, strftime
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
import os
import numpy as np
import shutil

In [3]:
!pip install mendelai-brat-parser

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for mendelai-brat-parser: filename=mendelai_brat_parser-0.0.11-py3-none-any.whl size=4945 sha256=d061dc3c5fb289b982388241b9abadd4b2c7fe53234b2b74bc98597d39c0b48d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/90/a7/ff/138853d8196095fec56e0a97779a96d754b98f169c063beca3
Successfully built mendelai-brat-parser
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


# data prepare

In [4]:
! python data_prepare_bmjl.py\
--label_dir './label_data/good'

process for (aspect, category)
a_list: ['feelings', 'expectancy', 'color', 'scene', 'purchase_behavior', 'size', 'design', 'price', 'fabric', 'style', 'audiences', 'quality', 'type', 'collocation', 'delivery']
finished process, result save to:  ./label_data/good/aspect_category.csv


In [5]:
! python data_prepare_bmjl.py\
--label_dir './label_data/bad'

process for (aspect, category)
a_list: ['size', 'scene', 'design', 'feelings', 'fabric', 'purchase_behavior', 'audiences', 'color', 'quality', 'type', 'price', 'style', 'collocation', 'expectancy', 'delivery']
finished process, result save to:  ./label_data/bad/aspect_category.csv


In [6]:
#merge data
good = './label_data/good/aspect_category.csv'
bad = './label_data/bad/aspect_category.csv'
df1 = pd.read_csv(good)
df2 = pd.read_csv(bad)
df_res = pd.concat([df1,df2])
df_res.to_csv('./aspect_category.csv')

In [7]:
import pandas as pd   
import numpy as np
import os
import shutil

#preprocess data
def write_txt(df,path):
    '''
    write back to txt
    '''
    #output txt file
    df = df.reset_index()
    with open(path,'a')as f:
        for i in range(len(df)):
            f.write("{} #### {}".format(df.loc[i,'text'].strip(),df.loc[i,'label']))
            f.write('\n')
            
            
def mkdir_rm(folder):
    '''
    make directory if not exists
    '''
    if os.path.exists(folder):
        shutil.rmtree(folder) 
    os.mkdir(folder)
    print ("<< path valid!")
    

def preprocess_data(input_file,output_path,over_sample=True):
    jsonObj = pd.read_csv(input_file)
    jsonObj = jsonObj[jsonObj['label']!='[]']
    print (jsonObj.head())
    
    #remove & remake the output folder 
    mkdir_rm(output_path)
    
    #generate tag.txt
    #a_list = ['consumer','zone','target','consequence','product','product_spec']
    #with open('tag.txt', 'w') as filehandle:
     #   filehandle.writelines("%s\n" % tag for tag in a_list)
    
    #train/test/val split
    train, validate, test = np.split(jsonObj.sample(frac=1), [int(.8*len(jsonObj)), int(.9*len(jsonObj))])
   
    print ("training size: ",train.shape)
    print ("test size: ",test.shape)
    print ("validate size: ",validate.shape)
    
    # write train/test/dev
    write_txt(train,os.path.join(output_path,'train.txt'))
    write_txt(test,os.path.join(output_path,'test.txt'))
    write_txt(validate,os.path.join(output_path,'dev.txt'))
    print ("<<<finish data preparing!")
    
input_file = './aspect_category.csv'
output_path = './data/tasd/bmjl'
preprocess_data(input_file,output_path,over_sample=False)

   Unnamed: 0  Unnamed: 0.1  sent_num  \
0           0             0         0   
1           1             0         0   
2           2             0         0   
3           3             0         0   
4           4             0         0   

                                                text  sent_start  sent_end  \
0  I like the shorts. They're comfortable. But I ...           0       313   
1  Absolutely love these shorts! The green color ...           0       215   
2  I ordered the purple tie dye color and was sen...           0       322   
3  I am 5’1 and around 110-112 lbs. I love these ...           0       453   
4  I am obsessed with these shorts. I’ve ordered ...           0       351   

   sent_len                                              label  \
0       313  [('comfortable', 'feelings'), ('being able to ...   
1       215  [('green color was perfect', 'color'), ('runni...   
2       322  [('Just wish I was given the correct color', '...   
3       453  [('I lo

# train

In [8]:
sess = sage.Session()

WORK_DIRECTORY = "./data"

# S3 prefix
prefix = "bmjl"

role = get_execution_role()

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
hyperparameters = {
    "task" : "tasd", 
    "dataset" : "bmjl", 
    "model_name_or_path" : "t5-base", 
    "paradigm": "extraction",
    "eval_batch_size" :"16",
    "train_batch_size" :"2",
    "learning_rate" :"3e-4",
    "num_train_epochs":"1",
    "n_gpu": "1"
}

In [9]:
entry_point = 'finetune.py'
source_dir = './'
git_config = None
role = get_execution_role()
framework_version = '1.7.1'
py_version='py36'
instance_type='ml.p3.2xlarge'
#instance_type='local_gpu'
instance_count=1

In [10]:
entry_point = 'finetune.py'
source_dir = './'
git_config = None
role = get_execution_role()
framework_version = '1.7.1'
py_version='py36'
instance_type='ml.p3.2xlarge'
#instance_type='local_gpu'
instance_count=1

In [11]:
estimator = PyTorch(
    entry_point = entry_point,
    source_dir = source_dir,
    git_config = git_config,
    role = role,
    debugger_hook_config=False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

In [12]:
inputs = {'tasd': data_location+'/tasd/'}

In [13]:
response = estimator.fit(inputs)

2022-08-17 02:58:35 Starting - Starting the training job...
2022-08-17 02:59:01 Starting - Preparing the instances for trainingProfilerReport-1660705115: InProgress
.........
2022-08-17 03:00:30 Downloading - Downloading input data...
2022-08-17 03:00:50 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-17 03:04:46,061 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-17 03:04:46,085 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-17 03:04:46,095 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-17 03:04:47,443 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2022-08-17 03:04:51 Training - Training image download completed. Traini

# deploy 

In [14]:
import sagemaker

instance_type = 'ml.m5.4xlarge'
role = sagemaker.get_execution_role()

In [15]:
s3_model = estimator.model_data 

In [16]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data=s3_model, 
                             role=role,
                             entry_point='inference.py', 
                             source_dir='./', 
                             framework_version='1.7.1', 
                             py_version='py36'
                ) # TODO set model_server_workers=1 to avoid torchhub bug

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

--------------!

In [17]:
from boto3.session import Session
import json

body = {"inputs": "I am pretty new to pickleball and finally decided to try out some different paddles."}

session = Session()
runtime = session.client("runtime.sagemaker")
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType="application/json",
    Body=json.dumps(body),
)
result = json.loads(response["Body"].read())
print (result)


{'result': '(pretty new to pickleball, scene); (finally decided to try out some different paddles, purchase_behavior)'}


In [18]:
%%time

predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

body = {"inputs": "I am pretty new to pickleball and finally decided to try out some different paddles."}

predictor.predict(body,initial_args={"ContentType":"application/json"})

CPU times: user 1.98 ms, sys: 11.8 ms, total: 13.8 ms
Wall time: 1.18 s


{'result': '(pretty new to pickleball, scene); (finally decided to try out some different paddles, purchase_behavior)'}

# batch transform

In [19]:
import csv
from sagemaker.s3 import S3Uploader,s3_path_join

# get the s3 bucket
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
sagemaker_session_bucket = sess.default_bucket()

#prepare data
dataset_csv_file = 'predict_0811.csv'
dataset_jsonl_file = "predict.jsonl"


i = 0
with open(dataset_csv_file, "r+") as infile, open(dataset_jsonl_file, "w+") as outfile:
    reader = csv.DictReader(infile)
    for row in reader:
        if i <5:
            json.dump({"inputs":row["0"]}, outfile)
            outfile.write('\n')
        i = i+1
                
# uploads a given file to S3.
input_s3_path = s3_path_join("s3://",sagemaker_session_bucket,"batch_transform/input")
output_s3_path = s3_path_join("s3://",sagemaker_session_bucket,"batch_transform/output")
s3_file_uri = S3Uploader.upload(dataset_jsonl_file,input_s3_path)

print(f"{dataset_jsonl_file} uploaded to {s3_file_uri}")

predict.jsonl uploaded to s3://sagemaker-us-east-1-726335585155/batch_transform/input/predict.jsonl


In [20]:
# create transformer to run a batch job
batch_job = pytorch_model.transformer(
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    output_path=output_s3_path,
    strategy='SingleRecord'
)

In [21]:
# starts batch transform job and uses S3 data as input
batch_job.transform(
    data=input_s3_path,
    content_type='application/json',    
    split_type='Line'
)

.......................................................Collecting transformers==4.6.0
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314476 sha256=a3a14471b331b7be42001fa26669b175c5ec2175fd50255c09610ca8487f69a6
  Stored in directory: /root/.cache/pip/wheels/17/a7/8b/a7e03881534e78558920ac68aaeca05180c0e2c3d11c4fce3b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895253 sha256=608c93633d2b3489c9711e524e9e25f58444430327c6b0b301192ee989532e50
  Stored in directory: /root/.cache/pip/wheels/4c/64/31/e9900a234b23fb3e9dc565d6114a9d6ff84a72dbdd356502b4
Successfully built jieba sacremoses
2022-08-17 03:44:28,984 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.3.1
TS Home: /opt/conda/lib/python3.6/site-packages
Curr

In [22]:
import json
from sagemaker.s3 import S3Downloader
from ast import literal_eval
# creating s3 uri for result file -> input file + .out
output_file = f"{dataset_jsonl_file}.out"
output_path = s3_path_join("s3://sagemaker-us-east-1-726335585155/batch_transform/output",output_file)

local_path = "output"  # Where to save the output locally

S3Downloader.download(output_path,local_path)

In [23]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [24]:
# Inspect the output

import os
import jsonlines
import json
from ast import literal_eval

output_file = f"{dataset_jsonl_file}.out"

batch_transform_result = []

path = os.path.join(local_path, output_file)
with open(path, "r") as f:
    for line in f:
        print (line)

{"result": "(VERY extra large, size)"}{"result": "(I really liked the look of this dress, style); (It just wasn\u2019t cute on me, feelings)"}{"result": "(Beautifully made, quality); (made in China, scene)"}{"result": "(I went up a size, size); (run a little small, size); (great quality, quality); (lined, fabric); (light weight, fabric); (very happy, feelings); (highly recommend, purchase_behavior)"}{"result": "(Not true to size, size)"}
